# Simple loop agent

In this example, we will create an loop agent that simply generate a description base on a same query and get all results

In other word, this is a "for loop" agent.

In [ ]:
import sys
# Change root directory
sys.path.append("../../src/")

In [2]:
from agent_design_pattern.agent import AgentMessage, LLMChain, BaseAgent
from agent_design_pattern.orchestration import LoopAgent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

/data/agent_design_pattern/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CasualOllamaChain(LLMChain):
    def __init__(self, model, system_prompt, user_prompt_template = "{query}", **kargs):
        super().__init__()
        self.llm = ChatOllama(model=model, reasoning=False, **kargs) if isinstance(model, str) else model
        self.prompt = ChatPromptTemplate([
            ("system", system_prompt),
            ("human", user_prompt_template)
        ])
        self.chain = self.prompt | self.llm | StrOutputParser()

    def invoke(self, message: AgentMessage, **kwargs) -> AgentMessage:
        response = self.chain.invoke(message.to_dict(), **kwargs)
        print(response)
        # Snippet to remove thinking
        if "</think>" in response and "<think>" not in response:
            response = "<think>" + response
        response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
        message.response = response

        message.execution_result = "success"
        return message


class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [ ]:
from typing import Generator


system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

def iterator(max_iteration: int) -> Generator[bool, None, None]:
    n = 0
    while True:
        yield n >= max_iteration
        n = (n + 1) % (max_iteration + 1)

chain = CasualOllamaChain(model="llama3.2:latest", system_prompt=system_prompt, user_prompt_template=user_prompt)
agent = Agent(chain, state_change_callback=state_callback)
loop_agent = LoopAgent(agent, iterator(max_iteration), state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=-1, temperature=1.2)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
O, fairest calm, thou dost deceive mine eyes,
With whispered silence and a gentle sigh,
As if the tempest's fury did despise
The peaceful dawn, where shadows softly lie.

Thy waves, they lap at yonder distant shore,
A soothing melody that doth entice once more,
To rest awhile and close mine eyelids tight,
And bid the world to slumber through the night.

But soft! what gentle breeze doth whisper low,
Of thunder's rumble, and a stormy woe?
The wind, it stirs, with troubled, fitful sighs,
As if the earth itself did feel the skies.

O, fleeting calm, thy peace doth soon depart,
And in its place, a tempest's wild heart.
Yet still I bask in thee, thy gentle might,
For in thy silence, lies a secret delight.
O, fairest dawn, that dost ascend on high,
With whispers soft, and gentle, languid sigh,
As if the night's dark veil had lifted wide,
And Night's sweet darkness fled before thy pride.

Thy rays, like golden arrows, pierce the air,
And with each breath, the morning's pr

In [6]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=1, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
O, fairest calmness that doth softly fall,
As whispered promise of the tempest's call,
When wind doth gently sway the leafy trees,
And sunbeams dance, in silvered ease.

The air, a canvas, soft and serene,
Doth wear the brushstrokes of a peaceful scene,
No discordant note doth pierce the ear,
As Nature's lullaby doth whisper near.

Yet, amidst this tranquil, easy reign,
A hidden force, like sleeping serpent, stains
Its power, and gathers strength to unleash
The fury that in secret calmness seethe.

Like patience worn, 'twas once a virtue true,
To bide the stormy tempests' wrath anew;
But now, as wisdom's voice doth whisper low,
"Beware, O heart, of calms that hide the woe."

O, peaceful hush, that veils the tempest's might,
How oft 'tis said, "The calm before the fight"!
For when the winds do howl and darkening clouds
Do gather fast, the storm doth fiercely crowds.

Thus let us heed this peaceful, silent call,
Lest we be lulled to sleep by a false fall;
And when th

In [7]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=3, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running


O, fairest calm, thou dost entwine thy gentle might
Amidst the whispers of the wind's deceitful sight,
A fleeting peace, that doth deceive mine eager eye,
As if the tempests' fury did but slowly pass on by.

Thy tranquil waves, they gently lap at yonder shore,
And in their quiet motion, all my worries are no more,
The thunder's rumble, soft and distant as a sigh,
Doth lull me into slumber, with a peaceful, heavy eye.

But lo! The trees, they stand, like sentinels of old,
Their leafy branches swaying, as if fate's thread were told,
In hushed converse, the breeze doth whisper secrets sweet,
Of trials to come, and passions that my soul shall meet.

And I, a mortal man, with fragile heart and feeble breath,
Do feel the tremors of a storm, in slumber's quiet death,
For in this fleeting calm, I sense the tempest's approach,
A darkening canvas, where the sun doth softly lose its cloak.

Yet still, I cling to thee, dear peace, as a lost child to a mother's breast,
For in thy gentle arms, my fe

In [ ]:
from typing import List


def custom_keep_logic(agent_name: str, message: AgentMessage, response: List[str]) -> List[str]:
    return False


query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=custom_keep_logic, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()